In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import io

In [6]:
splitdir = Path("/media/monaco/DATA1/case_study/24h_10mmMAX_OI/split")
convective = pd.read_csv(splitdir / "cluster_convective.csv", sep=";")
intermediate = pd.read_csv(splitdir / "cluster_intermediate.csv", sep=";")
stratiform = pd.read_csv(splitdir / "cluster_stratiform.csv", sep=";")

dates = pd.concat([
    convective.assign(NAME="convective"),
    intermediate.assign(NAME="intermediate"),
    stratiform.assign(NAME="stratiform")
])

In [7]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=9, random_state=42, shuffle=True)
n_split = 2
train_index, test_index = list(skf.split(dates, dates.NAME))[n_split]
val_index, train_index = np.split(train_index, [len(test_index)])

print(len(train_index), len(val_index), len(test_index))

314 45 45


# Result analysis

In [7]:
respath = Path("lightning_logs")
for model in respath.glob("[!_]*"):
    for split in model.glob("split_*"):
        for predpath in (split / "test_images").glob("_pred.png"):
            pred = Image.open(predpath).convert("L")

lightning_logs/unet/split_5
lightning_logs/unet/split_3
lightning_logs/ensemble_unet/split_1
lightning_logs/ensemble_unet/split_3
lightning_logs/sde_unet/split_5
lightning_logs/sde_unet/split_3


Out-of-distribution detection for regression on the dataset, where OoD are some extreme events hidden at train time.
We report the average performance and standard deviation for 9 random splits. All the  model performance is compared in terms of RMSE, AUROC, and AUPR.

In [6]:
# PAY ATTENTION THAT THIS IS DONE FOR THE n REPETITIONS OF THE PROBABILISTIC MODEL, EACH RETURNING A METRIC VALUE OVER WHICH WE CALCULATE THE MEAN AND STD
import torch

def calc_RMSE(pred_mean: torch.Tensor, target_img: torch.Tensor) -> float:
    """
    preds_mean and pred_std have shape (N, 1, H, W), target has shape (N, 1, H, W)
    """
    return torch.sqrt(((pred_mean - target_img) ** 2).mean())

def calc_AUROC(pred_variance: torch.Tensor, target: torch.Tensor) -> float:
    """
    preds_mean and pred_std have shape (N, 1, H, W), target has shape (N) and is a binary tensor saying if the nth event is an OoD one
    AUROC is calculated assigning a label to each prediction based on the average variance over all the pixels of the image
    """
    avg_variance = pred_variance.mean(dim=(1, 2, 3))
    curve_steps = 1000
    auroc = 0
    fprTemp = 1
    for delta in torch.arange(avg_variance.min(), avg_variance.max(), (avg_variance.max() - avg_variance.min()) / curve_steps):
        tpr = (avg_variance > delta).float().eq(target).float().mean()
        fpr = (avg_variance > delta).float().ne(target).float().mean()
        auroc += (fprTemp - fpr) * tpr
        fprTemp = fpr

def calc_AUPR_out(pred_variance: torch.Tensor, target: torch.Tensor) -> float:
    """
    preds_mean and pred_std have shape (N, 1, H, W), target has shape (N) and is a binary tensor saying if the nth event is an OoD one
    AUPR is calculated assigning a label to each prediction based on the average variance over all the pixels of the image
    """
    avg_variance = pred_variance.mean(dim=(1, 2, 3))
    curve_steps = 1000
    aupr = 0
    recallTemp = 1
    for delta in torch.arange(avg_variance.min(), avg_variance.max(), (avg_variance.max() - avg_variance.min()) / curve_steps):
        precision = (avg_variance > delta).float().ne(target).float().mean()
        recall = (avg_variance > delta).float().ne(target).float().mean()
        aupr += (recallTemp - recall) * precision
        recallTemp = recall

def calc_AUPR_in(pred_variance: torch.Tensor, target: torch.Tensor) -> float:
    target = target.eq(0).float()
    return calc_AUPR_out(pred_variance, target)

In [2]:
from utils.datasets import NWPDataset
from utils import io
import numpy as np
import torch
import pandas as pd
from pathlib import Path
class ExtremeDataset(NWPDataset):
    """
    Wrapper of NWPDataset returning the extreme events and the test events for the current split,
    together with a boolean label indicating if the event is extreme or not.
    """
    def __init__(self, split_idx, transform=None, seed=42,
                 input_path=Path("/media/monaco/DATA1/case_study/24h_10mmMAX_OI")):
        case_study = input_path.stem
        case_study_max, available_models, _, _, test_dates, indices_one, indices_zero, _, _, _ = io.get_casestudy_stuff(
			input_path, n_split=split_idx, case_study=case_study, ispadded=True,
			seed=seed)
 
        extreme_events = pd.concat([pd.read_csv(exev, header=None) for exev in input_path.glob('*extremeEvents.csv')])[0].values
        not_extreme_events = np.array([date for date in test_dates if date not in extreme_events])
        all_dates = np.concatenate([extreme_events, not_extreme_events])
        self.labels = torch.cat([torch.ones(len(extreme_events)), torch.zeros(len(not_extreme_events))]).long()

        X, Y, _, _ = io.load_data(input_path, all_dates, case_study_max, 
                                                       indices_one, indices_zero, available_models)
        super().__init__((torch.from_numpy(X), torch.from_numpy(Y).unsqueeze(1), 
                          torch.from_numpy(all_dates)), transform=transform)
    
    def __getitem__(self, index):
        return super().__getitem__(index) | {'isextreme_label': self.labels[index]}


In [9]:
x = '1.0333984375.1'


1.03339843751

In [11]:
x = '0.6759765625000006'
fixed_float(x)

In [19]:
df = pd.read_csv("data/24h_10mmMAX_OI/obs/data_extremeEvents/new_dataset/OI_20220727_regrid.csv", sep=";", header=None).to_numpy()
df1 = pd.read_csv("data/24h_10mmMAX_OI/obs/data/OI_20180606_regrid.csv", sep=";", header=None).to_numpy()
df.shape, df1.shape

((96, 116), (96, 116))

In [17]:
df.shape

(96, 116)

In [18]:
val.stem

'OI_20220727_regrid'

array([20220606, 20220701, 20220727, 20220906, 20220909, 20220928,
       20221001, 20221223, 20230413, 20230516, 20230703, 20230825,
       20230912, 20231113, 20231114, 20240318])

In [28]:
# path = Path("/home/smonaco/rainfall-prediction/data/24h_10mmMAX_OI/obs/data_extremeEvents/new_dataset")
# for val in path.glob("*csv"):
#     df = pd.read_csv(val, sep=";", header=None)
#     # for each value, if it is a string with 2 dots in it, remove the second one, joining the previous and the after part and parse the string as a float
#     def fixed_float(x):
#         if isinstance(x, str) and x.count(".") == 2:
#             return float(''.join(x.rsplit(".", 1)))
#         return x
#     df.map(fixed_float).to_csv(val, sep=";", index=False, header=False)

In [26]:
extreme_events[extreme_events < 20220000].shape

(30,)

In [42]:
obs = np.array([int(p.stem.split('_')[1]) for p in Path("data/24h_10mmMAX_OI/obs/data_extremeEvents/new_dataset/").iterdir()])

In [46]:
r = []
for nwp in ['bol00', 'c2200', 'c5m00', 'e1000']:
    r.append(np.array([int(p.stem.split('_')[1]) for p in Path("data/24h_10mmMAX_OI/models/data_extremeEvents/new_dataset/").glob(f'{nwp}*regrid.csv')]))
    print(len(set(r[-1]).intersection(pd.read_csv(input_path/'OI_20222024_10mmMAX_extremeEvents.csv', header=None)[0].values)))

16
16
16
16


In [35]:
extreme_events[extreme_events > 20220000]

array([20220523, 20220606, 20220701, 20220727, 20220906, 20220909,
       20220928, 20221001, 20221223, 20230413, 20230516, 20230703,
       20230825, 20230912, 20231113, 20231114, 20240318])

In [29]:
from pathlib import Path
input_path=Path("/home/smonaco/rainfall-prediction/data/24h_10mmMAX_OI")
case_study = input_path.stem
seed = 42
case_study_max, available_models, _, _, test_dates, indices_one, indices_zero, _, _, _ = io.get_casestudy_stuff(
			input_path, n_split=1, case_study=case_study, ispadded=True,
			seed=seed)
extreme_events = pd.concat([pd.read_csv(exev, header=None) for exev in input_path.glob('*extremeEvents.csv')])[0].values
not_extreme_events = np.array([date for date in test_dates if date not in extreme_events])
all_dates = np.concatenate([extreme_events, not_extreme_events])
X, Y, _, _ = io.load_data(input_path, all_dates, case_study_max, 
                                                       indices_one, indices_zero, available_models)

FileNotFoundError: File 'OI*20220906*.csv' does not exist

In [20]:
from training import get_args
args = get_args(['--network_model', 'unet', '-e', '1'])
from base_segmodel import _SegmentationModel

args.input_path = input_path
self = _SegmentationModel(**args.__dict__)

In [21]:
self.hparams.input_path

PosixPath('/home/smonaco/rainfall-prediction/data/24h_10mmMAX_OI')

In [23]:
case_study_max, available_models, train_dates, val_dates, test_dates, indices_one, indices_zero, mask, nx, ny = io.get_casestudy_stuff(
			self.hparams.input_path, n_split=self.hparams.n_split, case_study=self.hparams.case_study, ispadded=True,
			seed=self.hparams.seed
		)

In [25]:
out = io.load_data(self.hparams.input_path, test_dates, case_study_max, indices_one, indices_zero, available_models)

In [15]:
X, Y, _, _ = io.load_data(input_path, all_dates, case_study_max, indices_one, indices_zero, available_models)

IsADirectoryError: [Errno 21] Is a directory: '/home/smonaco/rainfall-prediction/data/24h_10mmMAX_OI'

In [13]:
extreme_ds = ExtremeDataset(1, input_path=input_path)

IsADirectoryError: [Errno 21] Is a directory: '/home/smonaco/rainfall-prediction/data/24h_10mmMAX_OI'

In [7]:
num_repetitions = 5

import pandas as pd
from pathlib import Path
from training import get_model
from utils.datasets import ExtremeDataset
from torch.utils.data import DataLoader
from tqdm.auto import trange, tqdm

results = pd.DataFrame(columns=["split", "model", "RMSE", "AUROC", "AUPR_out", "AUPR_in"])
for split in trange(9):
    extreme_ds = ExtremeDataset(split, input_path=Path('/home/smonaco/rainfall-prediction/data/24h_10mmMAX_OI'))
    extreme_dl = DataLoader(extreme_ds, batch_size=len(extreme_ds), shuffle=True)
    for model in tqdm(['unet', 'sde_unet', 'ensemble_unet', 'mcd_unet'], leave=False):
        checkpoint_path = f"lightning_logs/{model}/split_{split}/"
        if not checkpoint_path.exists():
            # print(f"\n>> Skipping model {model} for split {split}")
            continue

        ckpt = list(checkpoint_path.glob("*.ckpt"))
        assert len(ckpt) == 1
        ckpt = ckpt[0]
        model = get_model(model).load_from_checkpoint(ckpt)
        model.eval()
        print(model.device); foo
        
        x, y, ev_date, extremeev_label = next(iter(extreme_dl))
        x, y = x.to(model.device), y.to(model.device)
        pred_mean, pred_variance = model.multiple_eval(x, num_repetitions=num_repetitions)

        rmse = calc_RMSE(pred_mean, y)
        auroc = calc_AUROC(pred_variance, extremeev_label)
        aupr_out = calc_AUPR_out(pred_variance, extremeev_label)
        aupr_in = calc_AUPR_in(pred_variance, extremeev_label)

        results = pd.concat([results, pd.DataFrame({
            "model": model, "split": split, "RMSE": rmse, "AUROC": auroc, "AUPR_out": aupr_out, "AUPR_in": aupr_in
        })])

In [ ]:
from typing import Any
from easydict import EasyDict as edict
import torch

class Metrics:
    def __init__(self):
        self.models = []
        self.metrics = edict()
    
    def __call__(self, y_true, y_pred, model=None, split=None, **kwargs):
        if model not in self.models:
            self.models.append(model)
            self.metrics[model] = edict()
            self.metrics[model][split] = self.metric_func(y_true, y_pred, **kwargs)
        else:
            if split not in self.metrics[model]:
                self.metrics[model][split] = self.metric_func(y_true, y_pred, **kwargs)
            else:
                raise ValueError(f"Model {model} and split {split} already exists")
            

class RMSE(Metrics):
    def metric_func(self, y_true, y_pred):
        if isinstance(y_true, torch.Tensor):
            y_true = y_true.cpu().numpy()
        if isinstance(y_pred, torch.Tensor):
            y_pred = y_pred.cpu().numpy()
        return np.sqrt(np.mean((y_true - y_pred) ** 2))

class AUROC(Metrics):
    def metric_func(self, y_true, y_pred, isother=None):
        # isother is a bool array saying for each sample if it is OoD
        if isinstance(y_true, torch.Tensor):
            y_true = y_true.cpu().numpy()
        if isinstance(y_pred, torch.Tensor):
            y_pred = y_pred.cpu().numpy()
        


In [ ]:
out_of_class_dl = RainDataset() # contains image and ground truth of the OoD samples
out_of_class_dl = DataLoader(out_of_class_dl, batch_size=1, shuffle=False)

all_models = {
    'unet': [...],
    'mcd_unet': [...],
    'sde_unet': [...],

}
# This dictionary contains the models trained for each split in a list



for batch_id, data in enumerate(out_of_class_dl):
    image, target_img = data
    image = image.to(device)
    target_img = target_img.to(device)
    for model in all_models:
        for i, mi in enumerate(all_models[model]):
            mi.eval()
            with torch.no_grad():
                output = mi(image) # regression image of dimensions equal to the target
                RMSE.update(output, target_img, model=model, split=i)
                AUROC.update(output, target_img, model=model, split=i)
                AUPRC.update(output, target_img, model=model, split=i)
                MAE.update(output, target_img, model=model, split=i)
        AUROC.aggregate(model=model)
        AUPRC.aggregate(model=model)
        

            


# Xtreme events

In [8]:
input_path = Path('/media/monaco/DATA1/case_study')
n_split = 8
case_study = '24h_10mmMAX_OI'
input_path = input_path / case_study
use_extreme = False

case_study_max, available_models, train_dates, val_dates, test_dates, indices_one, indices_zero, mask, nx, ny = io.get_casestudy_stuff(
    input_path, n_split, case_study=case_study, ispadded=True,
    seed=42
)

print(len(train_dates), len(val_dates), len(test_dates), sum(len(x) for x in [train_dates, val_dates, test_dates]))

from sklearn.model_selection import train_test_split
test_dates1 = pd.concat([pd.read_csv(exev, header=None) for exev in input_path.glob('*extremeEvents.csv')])[0].values
train_dates = np.array(list(set(train_dates).union(val_dates).union(test_dates).difference(test_dates1)))
train_dates, val_dates = train_test_split(train_dates, train_size=315, random_state=42)
test_dates = test_dates1
print(len(train_dates), len(val_dates), len(test_dates), sum(len(x) for x in [train_dates, val_dates, test_dates])-17)




x_train, y_train, in_features, out_features = io.load_data(input_path, train_dates, case_study_max, indices_one, indices_zero, available_models)
x_val, y_val, in_features, out_features = io.load_data(input_path, val_dates, case_study_max, indices_one, indices_zero, available_models)
x_test, y_test, in_features, out_features = io.load_data(input_path, test_dates, case_study_max, indices_one, indices_zero, available_models)

315 45 45 405
315 59 48 405


AssertionError: File /media/monaco/DATA1/case_study/24h_10mmMAX_OI/models/bol00_20220606_0024_regrid.csv does not exist

In [14]:
for n, sp in zip(['tr', 'vl', 'ts'], [train_dates, val_dates, test_dates]):
    print(n)
    for d in sp:
        if not Path(f"/media/monaco/DATA1/case_study/24h_10mmMAX_OI/models/bol00_{d}_0024_regrid.csv").exists():
            print('>>', d)

tr
vl
ts
>> 20220606
>> 20220701
>> 20220727
>> 20220809
>> 20220906
>> 20220909
>> 20220928
>> 20221001
>> 20221223
>> 20230413
>> 20230516
>> 20230703
>> 20230825
>> 20230912
>> 20231113
>> 20231114
>> 20240318
